# Manual Bode Analysis Tool

This notebook provides interactive Bode analysis for HSData CSV files.

## Features:
- Extract frequency from filename
- Auto-detect VD input channel
- Manual time interval selection
- FFT analysis with configurable parameters
- VM/VD frequency response calculation
- Visualization of time and frequency domains
- Export results to CSV

## Block 1: Import Libraries and Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

Libraries imported successfully!


## Block 2: Data Reading and Frequency Extraction

In [3]:
def extract_frequency_from_filename(filename):
    """Extract frequency from filename (pure number format)"""
    # Extract number from filename (e.g., '100.csv' -> 100)
    match = re.search(r'(\d+)\.csv$', filename)
    if match:
        return float(match.group(1))
    else:
        raise ValueError(f"Cannot extract frequency from filename: {filename}")

def load_csv_data(file_path):
    """Load CSV data and extract frequency"""
    # Extract frequency from filename
    filename = Path(file_path).name
    frequency = extract_frequency_from_filename(filename)
    
    print(f"Loading file: {file_path}")
    print(f"Extracted frequency: {frequency} Hz")
    
    # Load CSV data
    df = pd.read_csv(file_path)
    
    # Extract VM and VD data
    vm_columns = [f'vm_{i}' for i in range(6)]
    vd_columns = [f'vd_{i}' for i in range(6)]
    
    vm_data = df[vm_columns].values
    vd_data = df[vd_columns].values
    
    print(f"Data shape: {vm_data.shape}")
    print(f"Time range: {df['index'].min()} to {df['index'].max()} samples")
    
    return vm_data, vd_data, frequency, df['index'].values

# Example usage (modify the path as needed)
# file_path = "01Data/02Processed_csv/0717_pi/100.csv"
# vm_data, vd_data, frequency, time_index = load_csv_data(file_path)

## Block 3: VD Channel Auto-Detection

In [4]:
def auto_detect_vd_channel(vd_data, threshold=1e-6):
    """Auto-detect which VD channel has input signal"""
    # Calculate RMS values for each VD channel
    vd_rms = np.sqrt(np.mean(vd_data**2, axis=0))
    
    print("VD Channel RMS values:")
    for i, rms in enumerate(vd_rms):
        print(f"  Channel {i}: {rms:.2e}")
    
    # Find channel with highest RMS (above threshold)
    max_channel = np.argmax(vd_rms)
    max_rms = vd_rms[max_channel]
    
    if max_rms > threshold:
        print(f"\nDetected VD input channel: {max_channel} (RMS: {max_rms:.2e})")
        return max_channel
    else:
        print(f"\nWarning: No VD channel above threshold {threshold}")
        return max_channel  # Return the highest one anyway

# Example usage
# vd_input_channel = auto_detect_vd_channel(vd_data)

## Block 4: Parameter Configuration

In [5]:
# ============================================================================
# CONFIGURABLE PARAMETERS - Modify these as needed
# ============================================================================

# Time interval selection
START_INDEX = 1000      # Starting index for data selection
DATA_LENGTH = 5000      # Number of data points to use

# FFT parameters
FFT_POINTS = None       # None = use data length, or specify manually
WINDOW_TYPE = 'none'    # 'none', 'hanning', 'hamming', 'blackman'
SAMPLING_FREQ = 100000  # Sampling frequency in Hz

# Display parameters
SHOW_TIME_DOMAIN = True
SHOW_FREQUENCY_DOMAIN = True

print("Parameter Configuration:")
print(f"  Start Index: {START_INDEX}")
print(f"  Data Length: {DATA_LENGTH}")
print(f"  FFT Points: {FFT_POINTS}")
print(f"  Window Type: {WINDOW_TYPE}")
print(f"  Sampling Frequency: {SAMPLING_FREQ} Hz")

Parameter Configuration:
  Start Index: 1000
  Data Length: 5000
  FFT Points: None
  Window Type: none
  Sampling Frequency: 100000 Hz


## Block 5: Data Preview and Time Domain Visualization

In [6]:
def plot_time_domain_data(vm_data, vd_data, time_index, start_idx, data_length, vd_input_channel):
    """Plot time domain data with selected interval"""
    
    # Calculate end index
    end_idx = start_idx + data_length
    
    # Create time axis in seconds
    time_sec = time_index / SAMPLING_FREQ
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Time Domain Data Analysis', fontsize=16)
    
    # Plot 1: Overall VM data (all channels)
    ax1 = axes[0, 0]
    for i in range(6):
        ax1.plot(time_sec, vm_data[:, i], label=f'VM_{i}', alpha=0.7)
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Voltage (V)')
    ax1.set_title('All VM Channels (Overall)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Highlight selected interval
    ax1.axvspan(time_sec[start_idx], time_sec[end_idx-1], alpha=0.3, color='yellow', label='Selected Interval')
    
    # Plot 2: Overall VD data (all channels)
    ax2 = axes[0, 1]
    for i in range(6):
        if i == vd_input_channel:
            ax2.plot(time_sec, vd_data[:, i], label=f'VD_{i} (Input)', linewidth=2, color='red')
        else:
            ax2.plot(time_sec, vd_data[:, i], label=f'VD_{i}', alpha=0.5)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Voltage (V)')
    ax2.set_title('All VD Channels (Overall)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Highlight selected interval
    ax2.axvspan(time_sec[start_idx], time_sec[end_idx-1], alpha=0.3, color='yellow')
    
    # Plot 3: Selected interval VM data
    ax3 = axes[1, 0]
    selected_time = time_sec[start_idx:end_idx]
    for i in range(6):
        ax3.plot(selected_time, vm_data[start_idx:end_idx, i], label=f'VM_{i}', alpha=0.7)
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Voltage (V)')
    ax3.set_title(f'Selected Interval VM Data ({start_idx} to {end_idx-1})')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Selected interval VD data
    ax4 = axes[1, 1]
    for i in range(6):
        if i == vd_input_channel:
            ax4.plot(selected_time, vd_data[start_idx:end_idx, i], label=f'VD_{i} (Input)', linewidth=2, color='red')
        else:
            ax4.plot(selected_time, vd_data[start_idx:end_idx, i], label=f'VD_{i}', alpha=0.5)
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Voltage (V)')
    ax4.set_title(f'Selected Interval VD Data ({start_idx} to {end_idx-1})')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print data statistics
    print("\nData Statistics for Selected Interval:")
    print("=" * 50)
    
    vm_selected = vm_data[start_idx:end_idx]
    vd_selected = vd_data[start_idx:end_idx]
    
    print("VM Channels:")
    for i in range(6):
        mean_val = np.mean(vm_selected[:, i])
        std_val = np.std(vm_selected[:, i])
        rms_val = np.sqrt(np.mean(vm_selected[:, i]**2))
        print(f"  Channel {i}: Mean={mean_val:.3e}, Std={std_val:.3e}, RMS={rms_val:.3e}")
    
    print("\nVD Channels:")
    for i in range(6):
        mean_val = np.mean(vd_selected[:, i])
        std_val = np.std(vd_selected[:, i])
        rms_val = np.sqrt(np.mean(vd_selected[:, i]**2))
        print(f"  Channel {i}: Mean={mean_val:.3e}, Std={std_val:.3e}, RMS={rms_val:.3e}")

# Example usage
# plot_time_domain_data(vm_data, vd_data, time_index, START_INDEX, DATA_LENGTH, vd_input_channel)

## Block 6: FFT Analysis and Response Calculation

In [7]:
def get_window_function(window_type, size):
    """Get window function for FFT"""
    if window_type == 'hanning':
        return np.hanning(size)
    elif window_type == 'hamming':
        return np.hamming(size)
    elif window_type == 'blackman':
        return np.blackman(size)
    else:
        return np.ones(size)  # No window

def calculate_frequency_response(vm_data, vd_data, start_idx, data_length, vd_input_channel, 
                                fft_points=None, window_type='none', sampling_freq=100000):
    """Calculate frequency response using FFT"""
    
    # Select data interval
    end_idx = start_idx + data_length
    vm_selected = vm_data[start_idx:end_idx]
    vd_selected = vd_data[start_idx:end_idx]
    
    # Get window function
    window = get_window_function(window_type, len(vm_selected))
    
    # Apply window to data
    vm_windowed = vm_selected * window[:, np.newaxis]
    vd_windowed = vd_selected * window[:, np.newaxis]
    
    # Perform FFT
    if fft_points is None:
        fft_points = len(vm_selected)
    
    vm_fft = np.fft.fft(vm_windowed, n=fft_points, axis=0)
    vd_fft = np.fft.fft(vd_windowed, n=fft_points, axis=0)
    
    # Calculate frequency axis
    freq_axis = np.fft.fftfreq(fft_points, 1/sampling_freq)
    
    # Calculate frequency response (VM/VD)
    # Use only the positive frequencies
    positive_freq_mask = freq_axis >= 0
    freq_positive = freq_axis[positive_freq_mask]
    
    # Get VD input channel data
    vd_input_fft = vd_fft[positive_freq_mask, vd_input_channel]
    
    # Calculate response for each VM channel
    responses = []
    for channel in range(6):
        vm_channel_fft = vm_fft[positive_freq_mask, channel]
        
        # Calculate complex response
        response = vm_channel_fft / vd_input_fft
        
        # Handle division by zero
        response[np.abs(vd_input_fft) < 1e-12] = 0
        
        # Calculate magnitude and phase
        magnitude = np.abs(response)
        phase_rad = np.angle(response)
        phase_deg = np.degrees(phase_rad)
        magnitude_db = 20 * np.log10(np.maximum(magnitude, 1e-12))
        
        responses.append({
            'channel': channel,
            'frequency': freq_positive,
            'magnitude': magnitude,
            'phase_deg': phase_deg,
            'magnitude_db': magnitude_db,
            'response_complex': response
        })
    
    return responses, freq_positive

# Example usage
# responses, freq_axis = calculate_frequency_response(
#     vm_data, vd_data, START_INDEX, DATA_LENGTH, vd_input_channel,
#     FFT_POINTS, WINDOW_TYPE, SAMPLING_FREQ
# )
# 
# # Step 5: Plot frequency domain data (if enabled)
# if SHOW_FREQUENCY_DOMAIN:
#     plot_frequency_response(responses, freq_axis, frequency, vd_input_channel)
# 
# # Step 6: Export results
# df_results = export_results_to_csv(responses, frequency)
# 
# print("\nAnalysis completed successfully!")

## Block 7: Frequency Domain Visualization

In [8]:
def plot_frequency_response(responses, freq_axis, frequency, vd_input_channel):
    """Plot frequency response results"""
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Frequency Response Analysis (Input Frequency: {frequency} Hz)', fontsize=16)
    
    # Plot 1: Magnitude response (linear scale)
    ax1 = axes[0, 0]
    for resp in responses:
        ax1.plot(freq_axis, resp['magnitude'], label=f'Channel {resp["channel"]}', alpha=0.7)
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Magnitude')
    ax1.set_title('Magnitude Response (Linear Scale)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, min(50000, freq_axis[-1]))  # Limit to 50kHz for better visibility
    
    # Plot 2: Magnitude response (dB scale)
    ax2 = axes[0, 1]
    for resp in responses:
        ax2.plot(freq_axis, resp['magnitude_db'], label=f'Channel {resp["channel"]}', alpha=0.7)
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Magnitude (dB)')
    ax2.set_title('Magnitude Response (dB Scale)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, min(50000, freq_axis[-1]))
    
    # Plot 3: Phase response
    ax3 = axes[1, 0]
    for resp in responses:
        ax3.plot(freq_axis, resp['phase_deg'], label=f'Channel {resp["channel"]}', alpha=0.7)
    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Phase (degrees)')
    ax3.set_title('Phase Response')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_xlim(0, min(50000, freq_axis[-1]))
    
    # Plot 4: Nyquist plot (polar plot)
    ax4 = axes[1, 1]
    for resp in responses:
        # Use only frequencies up to 10kHz for Nyquist plot
        freq_mask = freq_axis <= 10000
        real_part = np.real(resp['response_complex'][freq_mask])
        imag_part = np.imag(resp['response_complex'][freq_mask])
        ax4.plot(real_part, imag_part, label=f'Channel {resp["channel"]}', alpha=0.7)
    ax4.set_xlabel('Real Part')
    ax4.set_ylabel('Imaginary Part')
    ax4.set_title('Nyquist Plot (0-10kHz)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.axis('equal')
    
    plt.tight_layout()
    plt.show()
    
    # Print response at input frequency
    print(f"\nResponse at Input Frequency ({frequency} Hz):")
    print("=" * 60)
    
    # Find closest frequency bin
    freq_idx = np.argmin(np.abs(freq_axis - frequency))
    actual_freq = freq_axis[freq_idx]
    
    print(f"Closest frequency bin: {actual_freq:.1f} Hz")
    print(f"VD Input Channel: {vd_input_channel}")
    print("\nChannel Responses:")
    
    for resp in responses:
        magnitude = resp['magnitude'][freq_idx]
        phase = resp['phase_deg'][freq_idx]
        magnitude_db = resp['magnitude_db'][freq_idx]
        
        print(f"  Channel {resp['channel']}: Magnitude={magnitude:.4f} ({magnitude_db:.2f} dB), Phase={phase:.2f}°")

# Example usage
# plot_frequency_response(responses, freq_axis, frequency, vd_input_channel)

## Block 8: Results Export to CSV

In [9]:
def export_results_to_csv(responses, frequency, output_file=None):
    """Export frequency response results to CSV"""
    
    # Prepare data for export
    export_data = []
    
    for resp in responses:
        for i, freq in enumerate(resp['frequency']):
            export_data.append({
                'frequency': freq,
                'channel': resp['channel'],
                'magnitude': resp['magnitude'][i],
                'phase_deg': resp['phase_deg'][i],
                'magnitude_db': resp['magnitude_db'][i]
            })
    
    # Create DataFrame
    df_export = pd.DataFrame(export_data)
    
    # Generate output filename if not provided
    if output_file is None:
        output_file = f"bode_analysis_{frequency}Hz.csv"
    
    # Export to CSV
    df_export.to_csv(output_file, index=False)
    print(f"Results exported to: {output_file}")
    print(f"Total data points: {len(df_export)}")
    
    # Show sample of exported data
    print("\nSample of exported data:")
    print(df_export.head(10))
    
    return df_export

# Example usage
# df_results = export_results_to_csv(responses, frequency)

## Block 9: Complete Analysis Workflow

In [10]:
# ============================================================================
# COMPLETE ANALYSIS WORKFLOW
# ============================================================================
# Uncomment and modify the file path below to run the complete analysis

# # Step 1: Load data
# file_path = "01Data/02Processed_csv/0717_pi/100.csv"
# vm_data, vd_data, frequency, time_index = load_csv_data(file_path)
# 
# # Step 2: Auto-detect VD input channel
# vd_input_channel = auto_detect_vd_channel(vd_data)
# 
# # Step 3: Plot time domain data (if enabled)
# if SHOW_TIME_DOMAIN:
#     plot_time_domain_data(vm_data, vd_data, time_index, START_INDEX, DATA_LENGTH, vd_input_channel)
# 
# # Step 4: Calculate frequency response
# responses, freq_axis = calculate_frequency_response(
#     vm_data, vd_data, START_INDEX, DATA_LENGTH, vd_input_channel,
#     FFT_POINTS, WINDOW_TYPE, SAMPLING_FREQ
# )
# 
# # Step 5: Plot frequency domain data (if enabled)
# if SHOW_FREQUENCY_DOMAIN:
#     plot_frequency_response(responses, freq_axis, frequency, vd_input_channel)
# 
# # Step 6: Export results
# df_results = export_results_to_csv(responses, frequency)
# 
# print("\nAnalysis completed successfully!")

## Usage Instructions

1. **Load your data**: Modify the file path in Block 9
2. **Adjust parameters**: Modify the parameters in Block 4
3. **Run analysis**: Execute Block 9
4. **Review results**: Check the plots and exported CSV
5. **Iterate**: Adjust parameters and re-run as needed

### Parameter Tuning Tips:

- **START_INDEX**: Choose a stable region of the signal
- **DATA_LENGTH**: Use power of 2 for better FFT performance
- **WINDOW_TYPE**: Use 'none' for transient signals, 'hanning' for steady-state
- **FFT_POINTS**: Leave as None for automatic, or specify for zero-padding